In [2]:
from datetime import datetime, timedelta, date
import urllib.parse

peopleexport = r"C:\Users\ralph\Downloads\newmarketchurch-people-02-15-2024.xlsx"

ncc_url = r"https://newmarketchurch.breezechms.com"

# Set up Breeze API wrapper
from breeze_chms_api import breeze

# Initialize API 
import config
breeze_api = breeze.breeze_api(breeze_url=ncc_url,
                               api_key=config.api_key)


import numpy as np
import pandas as pd

def make_link(row):
    # print(row)
    url = ncc_url + r'/people/view/' + str(row['Breeze ID'])
    name = row['Last Name']
    if isinstance(row['First Name'], str):
        name = row['First Name'] + " " + name
    return f'<a href="{url}">{name}</a>'

# Read in the downloaded Excel file 
people = pd.read_excel(peopleexport, na_filter=False)

# people.insert(loc = 1, column = 'link', value = people.apply(make_link, axis=1))
# people.style.format({'link': make_link})
# people['link'] = people.apply(make_link, axis=1)
print('{count} people in input.'.format(count = len(people.index)))

1906 people in input.


In [7]:
# display(breeze_api.get_person_details(person_id = 47805612))

{'id': '47805612',
 'first_name': 'Cynthia',
 'force_first_name': 'Cynthia',
 'nick_name': '',
 'middle_name': '',
 'maiden_name': '',
 'last_name': 'Whitney',
 'thumb_path': '',
 'path': 'img/profiles/generic/gray.png',
 'street_address': '291 Wadleigh Falls Road',
 'city': 'Lee',
 'state': 'NH',
 'zip': '03861',
 'details': {'person_id': '47805612',
  '0': 'on',
  '': 'on',
  '1515006295': [{'name': 'Friend of NCC - Donor', 'value': '99'}],
  '1494581201': [{'field_type': 'phone',
    'phone_number': '(603) 765-1438',
    'phone_type': 'home',
    'do_not_text': '0',
    'is_private': '0',
    'people_meta_id': '1090977876'}],
  '487851450': [{'address': 'connemara3@aol.com',
    'is_primary': '1',
    'allow_bulk': '1',
    'is_private': '0',
    'field_type': 'email_primary'}],
  '589696826': [{'field_type': 'address_primary',
    'street_address': '291 Wadleigh Falls Road',
    'city': 'Lee',
    'state': 'NH',
    'zip': '03861',
    'longitude': '',
    'latitude': '',
    'is_p

In [6]:
# Check the City in the Address
def clean(word):
    if word == 0:
        return 'unknown'
    return word.strip().title()

print("City needs to be cleaned up:")
# display(people.loc[people['City'] != people['City'].apply(clean)][['Breeze ID', 'City']])
fix_city = people.loc[people['City'] != people['City'].apply(clean)]['Breeze ID']
print(len(fix_city))
display(fix_city)
for person in fix_city.tolist()[0:1]:
    profile = breeze_api.get_person_details(person_id = person)
    details = profile['details']
    # print('\n profile: ', profile)
    address = {k: v for k, v in details.items() if isinstance(v, list) and len(v) > 0 and 'field_type' in v[0] and v[0]['field_type'] == 'address_primary'}
  
    # print(address)
    address_key = [*address][0]
    paddress = address[address_key][0]
    # print(paddress)
    
    paddress['city'] = clean(paddress['city'])
    paddress['street_address'] = clean(paddress['street_address'])
    # print(address)
    details[address_key] = [paddress]
    # print('\n modified: ', details)
    profile['details'][address_key][0]['city'] = paddress['city']
    profile['details'][address_key][0]['street_address'] = paddress['street_address']
    # print('\n profile: ', profile)
    # print(profile['city'], update)

    # update_person doesn't seem to update the profile
    
    result = breeze_api.update_person(person_id = person, fields_json = [details])
    # print('\n result: ', result)
    # print(len(profile), type(profile), profile['city'], profile)

City needs to be cleaned up:
151


1       30395820
24      30395884
25      30395886
29      30395902
42      30395946
          ...   
1831    47397912
1843    47527927
1875    47715579
1879    47727369
1894    47831829
Name: Breeze ID, Length: 151, dtype: int64

In [3]:
# Some people don't have a first name. Organizations shouldn't, so that's not a problem.
# Is there a better way to know if a profile belongs to an organization?
# A name that starts with a ' ' is probably a problem.

columns_to_check = [
    'First Name',
    'Last Name',
]

def show(frame, col, rule):
    if (len(frame.index) > 0):
        print('column {col} and rule {rule}'.format(col = col, rule = rule))
        display(frame)

for col in columns_to_check:
# print(type(people['First Name']))
# people[pd.isna(people['First Name'])]
    show(people[people[col].apply(lambda x : not isinstance(x, str))], col, 'should be a string')
    show(people[people[col].apply(lambda x : isinstance(x, str) and x.startswith(' '))], col, 'should start with a letter')
    show(people[people[col].apply(lambda x : isinstance(x, str) and not x == x.strip())], col, 'should start and end with a letter')

In [4]:
# TODO: Should orgs have an 'Org...' tag?

In [5]:
people['First Name'] = people['First Name'].replace(np.nan, '')
# people = people.iloc[0:25]
# print(people)

In [6]:
from fuzzywuzzy import fuzz 

# print(people['First Name'].loc[0:24])
matches = []
for row in range(0, len(people)): 
    # print(row)
    # print(people.iloc[row]['First Name'], people.iloc[row]['Last Name'])
    bestmatch = (0, '') 
    for other in [people.iloc[i] for i in range(len(people)) if i != row]:
        # print(people.iloc[row]['First Name'], people.iloc[row]['Last Name'])
        fcomp = fuzz.ratio(people.iloc[row]['First Name'], other['First Name'])
        lcomp = fuzz.ratio(people.iloc[row]['Last Name'], other['Last Name'])
        # TODO: Does this calculation work for orgs where the First Name is blank?
        match = fcomp * lcomp / 100
        if match > bestmatch[0]:
            bestmatch = (match, other['First Name'] + ' ' + other['Last Name'])
    # print(bestmatch)
    matches.append(bestmatch)
    # print(people['First Name'].loc[i for i in range(len(people)) if i != row])
# print(people.iloc[0])
# for p in people['First Name'].tolist():
# print(matches)


KeyboardInterrupt: 

In [ ]:
people['Best Match ratio'] = [m[0] for m in matches]
people['Best Match name'] = [m[1] for m in matches]

In [ ]:
allnames = (people['First Name'].replace(np.nan, '') + ' ' + people['Last Name']).tolist()
# print(allnames)
for name in allnames:
    if not isinstance(name, str):
        print(name)

In [ ]:
# from operator import itemgetter 
# from fuzzywuzzy import process

# Ratios = [process.extract(x, allnames) for x in allnames]
# maxes = [max(ratio, key = itemgetter(1)) for ratio in Ratios if ratio[0][1] != 100]
# print(maxes)


In [ ]:
display(people[people['Best Match ratio'] > 90])

In [ ]:
# Set up Breeze API wrapper
from breeze_chms_api import breeze

# Initialize API 
breeze_api = breeze.breeze_api(breeze_url='https://newmarketchurch.breezechms.com',
                               api_key='8dfd0a0d7f5aaec745a73542f58eb8ba')

for index, person in people.iterrows():
# for index, person in people.iterrows():
    if 44241778 == person['Breeze ID'] or index < 2:
        person = breeze_api.get_person_details(person_id = person['Breeze ID'])
        # print(person['details'])
        # for key, val in person.items():
        #     if key == 'family' and len(val) > 0:
        #         print(key, val)
        family = person['family']
        if len(family) > 0:
            print(family)
        # for key, val in person['details'].items():
        #     print(key, val)